# theano共享变量
上节介绍了theano的符号变量及计算图的运行机制，本节将介绍另一个重要概念——共享变量。与符号变量稍有不同的是，共享变量有明确的值，可以被get/set，并且在使用它的函数间共享。

In [1]:
from __future__ import print_function

import numpy as np
import theano
import theano.tensor as T

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


共享变量的类型可以根据初始化方式自动推导

In [2]:
shared_var = theano.shared(np.array([[1, 2], [3, 4]], dtype=theano.config.floatX))
print(shared_var.type())

<CudaNdarrayType(float32, matrix)>


我们可以用set_value设置它的值，用get_value获取它的值

In [3]:
shared_var.set_value(np.array([[3, 4], [2, 1]], dtype=theano.config.floatX))
print(shared_var.get_value())

[[ 3.  4.]
 [ 2.  1.]]


与符号变量类似的地方是，我们可以为它定义函数关系。注意到shared_var已经有确切的值，所以它不必作为函数输入参数的一部分，因为theano已经隐式地将其作使用表达式shared_squared的函数的输入。

In [4]:
shared_squared = shared_var**2
function_1 = theano.function([], shared_squared)
print(function_1())

[[  9.  16.]
 [  4.   1.]]


## updates
可以使用`theano.function`的`updates`参数来自动更新共享变量的值

In [5]:
subtract = T.matrix('subtract')
# updates以一个字典为输入，字典的键位共享变量，值为更新的表达式
# 这里updates被设置为shared_var = shared_var - subtract
function_2 = theano.function([subtract], shared_var, updates={shared_var: shared_var - subtract})
print("shared_var before subtracting [[1, 1], [1, 1]] using function_2:")
print(shared_var.get_value())
# 减去矩阵[[1, 1], [1, 1]]
function_2(np.array([[1, 1], [1, 1]], dtype=theano.config.floatX))
print("shared_var after calling function_2:")
print(shared_var.get_value())
# 注意到共享变量被所有使用它的函数共享， 由于它的值已经改变， 因此function_1()的输出也将改变
print("New output of function_1() (shared_var**2):")
print(function_1())

shared_var before subtracting [[1, 1], [1, 1]] using function_2:
[[ 3.  4.]
 [ 2.  1.]]
shared_var after calling function_2:
[[ 2.  3.]
 [ 1.  0.]]
New output of function_1() (shared_var**2):
[[ 4.  9.]
 [ 1.  0.]]


## 如何获取共享变量的shape
共享变量的shape是个`TensorVariable`，如果直接print的话是无法查看具体的值的

In [6]:
print("type of shared_var:{}".format(type(shared_var.shape)))
print("shared_var.shape:{}".format(shared_var.shape))

type of shared_var:<class 'theano.tensor.var.TensorVariable'>
shared_var.shape:Shape.0


定义一个函数来查看shape的过程太过繁琐，解决的办法是使用eval函数

In [7]:
print("shared_var.shape.eval():{}".format(shared_var.shape.eval()))

shared_var.shape.eval():[2 2]


## 共享变量构造器参数`borrow`的作用
borrow参数指定了在定义共享变量时是深拷贝还是浅拷贝。如果`borrow`为`False`，则是深拷贝，反之则为浅拷贝。接下来我们定义三个共享变量，分别对应`borrow`取默认值，False，True

In [8]:
np_array = np.ones(2, dtype='float32')

s_default = theano.shared(np_array)
s_false   = theano.shared(np_array, borrow=False)
s_true    = theano.shared(np_array, borrow=True)

接着我们改变变量`np_array`

In [9]:
np_array += 1 # now it is an array of 2.0 s

print(s_default.get_value())
print(s_false.get_value())
print(s_true.get_value())

[ 1.  1.]
[ 1.  1.]
[ 1.  1.]


由结果我们看到，当`borrow`取默认值时，使用的初始化方式是深拷贝；当`borrow=False`时使用的方式也是深拷贝；当`borrow=True`时，要分情况来讨论，如果运行代码的设备是CPU，那么初始化方式将是浅拷贝，输出结果则是[2, 2]， 如果设备是GPU，那么`borrow`将失去作用，也就是说不管它是`True`还是`False`，初始化方式都将是深拷贝（因为需要将数据从CPU拷贝到GPU来运行）。